In [1]:
# Importando las librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importando el conjunto de datos
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Codificación de datos categóricos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])  # Codificando la columna 'Geography'
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])  # Codificando la columna 'Gender'



In [4]:
# Aplicando OneHotEncoder para la columna 'Geography'
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'),[1])], remainder='passthrough')
# hago drop = 'first' para evitar el problema de las variables ficticias
# No lo hago para la columna género porque ya tengo 0 0 1 según sea mujer u hombre.
# Si lo hubiese aplicado -> OneHotEncoder(drop='first'),[1,2])
X = np.array(ct.fit_transform(X))

In [5]:
# Dividiendo el conjunto de datos en conjunto de entrenamiento y conjunto de prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Escalado de características (Feature Scaling)
# Fundamental en Redes Neuronales
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## PARTE 2- CONSTRUIR LA RNA (Red Neuronal Artificial)

# Importar Keras y librerías adicionales

In [7]:
from tensorflow import keras
from tensorflow.keras.models import Sequential #Para inicializar los parámetros de la Red Neuronal
from tensorflow.keras.layers import Dense # Para declarar y crear las capas intermedias de la Red Neuronal
from tensorflow.keras.layers import Dropout # Para crear la capa de dropout (de olvido) - útil cuando tenemos mucha varianza - Evita overfitting desactivando algunas neurones de la capa inmediatamente anterior 

## PARTE 3- CONSTRUIR LA RNA

In [8]:
# Inicializar la RNA
classifier = Sequential()


In [9]:
# Añadir capas de entrada y primeras capa oculta
# Paso 1 - Inicializar los pesos aleatoriamente con valores cercanos a 0 (pero no 0)
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu", input_dim = 11 ))
# Buena regla para los units (nodos) -> Media entre los nodos de entrada y los nodos de salida ( (11 + 1)/2 = 6)

C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# añadimos una capa de Dropout
classifier.add(Dropout(rate=0.1))

In [13]:
# Añadir la siguiente capa oculta
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu"))
# La siguiente capa oculta ya sabe lo que espera de la primera capa -> input_dim no es necesario
# podemos dejar el mismo número de nodos en la siguiente capa oculta. Tb la misma función


In [14]:
# añadimos una capa de Dropout
classifier.add(Dropout(rate=0.1))

In [15]:
# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = "sigmoid"))
# en la capa de salida quiero una probabilidad -> función de activació: función sigmoide

In [16]:
# Compilar la RNA
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [17]:
# Ajustamos la RNA al conjunto de entrenamiento (Paso 2)
classifier.fit(X_train,y_train, batch_size=10, epochs = 100)
#batch_size=10 -> procesar 10 elementos -> Corregir los pesos -> procesar 10 elementos -> Corregir los pesos (más óptimo)
#epochs=100 -> Dará 100 pasadas a nuestros datos

Epoch 1/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7958 - loss: 0.4955
Epoch 2/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.4379
Epoch 3/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.4342
Epoch 4/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.4315
Epoch 5/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.4301
Epoch 6/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.4322
Epoch 7/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.4314
Epoch 8/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8016 - loss: 0.4280
Epoch 9/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8200 - loss: 0.4288
Epoch 10/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8215 - loss: 0.4282
Epoch 11/100
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8229 - loss: 0.4275
Epoch 12/100
800/800 ━━━━━━━━━━━━━━━━━━━━

# Parte 3 - Evaluar el modelo y calcular predicciones  finales

In [18]:
# Predicción de los resultados con el conjunto de Testing
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [19]:
# Elaborar una matriz de confusión
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score, recall_score, f1_score
cm = confusion_matrix(y_test, y_pred)

In [20]:
cm

array([[1560,   35],
       [ 281,  124]], dtype=int64)

In [21]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.842


In [22]:
precision = precision_score(y_test, y_pred)
print("Precisión: ", precision)

Precisión:  0.779874213836478


In [23]:
recall = recall_score(y_test, y_pred)
print("Sensibilidad: ", recall)

Sensibilidad:  0.30617283950617286


In [24]:
f1 = f1_score(y_test, y_pred)
print("F1:", f1)

F1: 0.4397163120567376


# Tarea

In [25]:
nuevo_cliente = np.array([[
    600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000
]], dtype=object)

In [26]:
# 1. label encoders (ojo a los índices: aquí geography está en 1 y gender en 2)
nuevo_cliente[:, 1] = labelencoder_X_1.transform(nuevo_cliente[:, 1])
nuevo_cliente[:, 2] = labelencoder_X_2.transform(nuevo_cliente[:, 2])

In [27]:
# 2. one-hot
nuevo_cliente_encoded = ct.transform(nuevo_cliente)
 # mismo corte que con X

In [28]:
# 3. escalado
nuevo_cliente_scaled = sc.transform(nuevo_cliente_encoded)


In [29]:
# 4. predicción

pred_nueva= classifier.predict(nuevo_cliente_scaled)
pred_nueva = (pred_nueva>0.5)
print(pred_nueva)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[False]]


# Parte 4 - Evaluar, mejorar y ajustar la RNA

## Evaluar la RNA

In [30]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

In [31]:
def build_classifier():
    # Inicializar la RNA
    classifier = Sequential()
    # Añadir capas de entrada y primeras capa oculta
    # Paso 1 - Inicializar los pesos aleatoriamente con valores cercanos a 0 (pero no 0)
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu", input_dim = 11 ))
    # Añadir la siguiente capa oculta
    
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu"))
    # Añadir la capa de salida
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = "sigmoid"))
    # en la capa de salida quiero una probabilidad -> función de activació: función sigmoide
    # Compilar la RNA
    classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

    #Devolver el clasificador
    return classifier


In [32]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator= classifier,X = X_train, y= y_train,n_jobs= -1, cv =10,verbose=1)
# n_jobs=-1 -> toma la mayor cantidad de núcleros de nuestro ordenador
# verbose=1 -> Nos indica la evolución. Por defecto verbose= 0

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  2.5min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.5min finished


In [33]:
accuracies

array([0.84   , 0.84   , 0.87375, 0.8575 , 0.84875, 0.83125, 0.8675 ,
       0.8575 , 0.8125 , 0.8725 ])

In [34]:
mean = accuracies.mean()
variance = accuracies.std()
print("Valor medio: ", mean)
print("Varianza: ", variance)

Valor medio:  0.850125
Varianza:  0.01856954832514783


# Mejorar la RNA

## Regularización de Dropout para evitar el overfitting

## Ajustar la RNA

In [35]:
from sklearn.model_selection import GridSearchCV #inckluye GridSearch

In [42]:
def build_classifier(optimizer = 'adam'):
    # Inicializar la RNA
    classifier = Sequential()
    # Añadir capas de entrada y primeras capa oculta
    # Paso 1 - Inicializar los pesos aleatoriamente con valores cercanos a 0 (pero no 0)
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu", input_dim = 11 ))
    # Añadir la siguiente capa oculta
    
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = "relu"))
    # Añadir la capa de salida
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = "sigmoid"))
    # en la capa de salida quiero una probabilidad -> función de activació: función sigmoide
    # Compilar la RNA
    classifier.compile(optimizer, loss = "binary_crossentropy", metrics=["accuracy"])

    #Devolver el clasificador
    return classifier

In [43]:
classifier = KerasClassifier(build_fn = build_classifier) # Ahora build_fn => model
#Eliminamos batch_size = 10 y epochs = 100 pòrque ya no serán parámetros fijos

In [44]:
# definimos un diccionario con los parámetros
parameters= {
    'batch_size' : [25, 32],
    'epochs' : [100, 500],
    'optimizer' : ['adam', 'rmsprop']
    
}

In [45]:
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)

# scoring es la métrica. En este caso precisión. mPodría ser tb error cuadrático medio

grid_search = grid_search.fit(X_train, y_train) # el propio objeto se ajustará a sí mismo


Epoch 1/100


C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7971 - loss: 0.5483
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7971 - loss: 0.4348
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7971 - loss: 0.4287
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7971 - loss: 0.4247
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7971 - loss: 0.4220
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8035 - loss: 0.4191
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8219 - loss: 0.4171
Epoch 8/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8239 - loss: 0.4160
Epoch 9/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8294 - loss: 0.4145
Epoch 10/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8299 - loss: 0.4134
Epoch 11/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8300 - loss: 0.4124  
Epoch 12/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/st

C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7967 - loss: 0.5594
Epoch 2/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7967 - loss: 0.4351
Epoch 3/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7967 - loss: 0.4300
Epoch 4/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7967 - loss: 0.4271
Epoch 5/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7967 - loss: 0.4249
Epoch 6/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7967 - loss: 0.4221
Epoch 7/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8047 - loss: 0.4196
Epoch 8/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8203 - loss: 0.4176
Epoch 9/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8249 - loss: 0.4162
Epoch 10/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8264 - loss: 0.4152
Epoch 11/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8293 - loss: 0.4144
Epoch 12/100
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

...

C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7922 - loss: 0.6338
Epoch 2/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7944 - loss: 0.4690
Epoch 3/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4356
Epoch 4/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4281
Epoch 5/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4241
Epoch 6/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4208
Epoch 7/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4179
Epoch 8/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4148
Epoch 9/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4112
Epoch 10/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7944 - loss: 0.4077
Epoch 11/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8006 - loss: 0.4038
Epoch 12/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\srmjf\anaconda3\envs\dl\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7960 - loss: 0.5829
Epoch 2/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7969 - loss: 0.4403
Epoch 3/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7969 - loss: 0.4316
Epoch 4/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7969 - loss: 0.4278
Epoch 5/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7969 - loss: 0.4249
Epoch 6/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7969 - loss: 0.4229
Epoch 7/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7969 - loss: 0.4205
Epoch 8/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8106 - loss: 0.4184
Epoch 9/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8199 - loss: 0.4178
Epoch 10/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8257 - loss: 0.4163
Epoch 11/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8251 - loss: 0.4154  
Epoch 12/500
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/st

KeyboardInterrupt: 

In [ ]:
best_parameters = grid_search.best_params_ #añadimos una barra baja a la derecha porque son los parámentros que devuelve el algoritmo
best_accuracy = grid_search.best_scores_